In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

# 1.4: Unconformity relationships


In [ ]:
# sphinx_gallery_thumbnail_number = 2

根据提供的教程代码（ch1_4_onlap_relations.ipynb）以及数据点文件，这个地质模型构建了一个典型的**超覆（Onlap）构造**，通常用于模拟沉积盆地边缘或俯冲带的几何形状。

以下是该地层模型可能的样子及其地质演化顺序（从老到新）：

### 1. 整体几何结构
模型在空间上大致分为“左侧”和“右侧”，中间由一个陡峭的侵蚀面隔开：
*   **左侧**：主要是古老的基底岩石。
*   **右侧**：是一个被年轻沉积物填充的盆地或斜坡。
*   **顶部**：被一个平坦的海底表面截断。

### 2. 地层序列（从下往上/从老到新）

根据代码中 `gp.add_structural_group` 的定义顺序和关系类型：

1.  **基底序列 (Left Series / Basement)**
    *   **地层**: `rock3`
    *   **位置**: 主要分布在模型的左侧（X轴较小的区域）。
    *   **特征**: 这是模型中最古老的岩石，构成了地质体的基础。

2.  **超覆面序列 (Onlap Series / Erosion)**
    *   **地层**: `onlap_surface`
    *   **特征**: 这是一个**侵蚀面（Unconformity）**。它像一个陡峭的斜坡或峡谷壁，切穿了基底（`rock3`）。根据数据点，这个面在模型中部向右侧倾斜（Dip 较大）。

3.  **右侧序列 (Right Series / Onlap)**
    *   **地层**: `rock2` (下), `rock1` (上)
    *   **关系**: **超覆 (Onlap)**。
    *   **特征**: 这些是年轻的沉积层，沉积在 `onlap_surface` 形成的“坑”或斜坡右侧。
    *   **几何形态**: 关键在于“超覆”关系。这意味着 `rock1` 和 `rock2` 的层位是水平或缓倾斜的，但它们**终止于**陡峭的 `onlap_surface` 上。它们并没有像褶皱一样弯曲平行于基底，而是像填土一样“靠”在斜坡上，层厚向左侧逐渐变薄直至尖灭。

4.  **海底序列 (Seafloor Series / Erode)**
    *   **地层**: `seafloor`
    *   **关系**: **侵蚀 (Erode)**。
    *   **特征**: 这是最年轻的层，位于模型的最顶部（Z轴约 -200m 处）。它水平地切过下面的所有地层（基底和超覆沉积物），代表了当前的海底地形。

### 3. 视觉想象
想象一个陡峭的水下悬崖（左边是基底 `rock3`）。随着时间的推移，沉积物（`rock2`, `rock1`）在悬崖右侧的低洼处逐层堆积，这些沉积层在接触到悬崖壁时终止。最后，整个区域被海水覆盖，形成了一个平坦的海底（`seafloor`）。

代码末尾的 `mask_matrix` 绘图正是为了展示这些不同序列在空间上的有效区域（即哪些地方是基底，哪些地方被超覆层填充）。

Importing gempy


In [ ]:
import os

import gempy_viewer as gpv

# Aux imports
import numpy as np
from gempy_engine.config import AvailableBackends

import gempy as gp

np.random.seed(1515)

# 修复 OMP: Error #15 冲突
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

We import a model from an existing folder, representing a subduction
zone with onlap relationships.


In [ ]:
data_path = os.path.abspath("..")

geo_model: gp.data.GeoModel = gp.create_geomodel(
    project_name="Onlap_relations",
    extent=[-200, 1000, -500, 500, -1000, 0],
    resolution=[50, 50, 50],
    refinement=6,
    importer_helper=gp.data.ImporterHelper(
        path_to_orientations=data_path + "/data/tut_ch1-4_orientations.csv",
        path_to_surface_points=data_path + "/data/tut_ch1-4_points.csv",
    ),
)

gp.set_topography_from_random(grid=geo_model.grid, d_z=np.array([-600, -100]))

In [ ]:
gpv.plot_2d(geo_model)

Raw structural frame


In [ ]:
geo_model.structural_frame

In [ ]:
geo_model.input_transform.apply_anisotropy(gp.data.GlobalAnisotropy.NONE)
gp.add_structural_group(
    model=geo_model,
    group_index=0,
    structural_group_name="seafloor_series",
    elements=[geo_model.structural_frame.get_element_by_name("seafloor")],
    structural_relation=gp.data.StackRelationType.ERODE,
)

gp.add_structural_group(
    model=geo_model,
    group_index=1,
    structural_group_name="right_series",
    elements=[
        geo_model.structural_frame.get_element_by_name("rock1"),
        geo_model.structural_frame.get_element_by_name("rock2"),
    ],
    structural_relation=gp.data.StackRelationType.ONLAP,
)

gp.add_structural_group(
    model=geo_model,
    group_index=2,
    structural_group_name="onlap_series",
    elements=[geo_model.structural_frame.get_element_by_name("onlap_surface")],
    structural_relation=gp.data.StackRelationType.ERODE,
)

gp.add_structural_group(
    model=geo_model,
    group_index=3,
    structural_group_name="left_series",
    elements=[geo_model.structural_frame.get_element_by_name("rock3")],
    structural_relation=gp.data.StackRelationType.BASEMENT,
)

gp.remove_structural_group_by_name(model=geo_model, group_name="default_formation")

# Final structural frame
geo_model.structural_frame

In [ ]:
s = gp.compute_model(gempy_model=geo_model, engine_config=gp.data.GemPyEngineConfig(backend=AvailableBackends.PYTORCH))

In [ ]:
gpv.plot_2d(geo_model, show_data=True)

In [ ]:
gpv.plot_3d(
    model=geo_model,
    show_surfaces=True,
    show_data=True,
    image=True,
    show_topography=True,
    kwargs_plot_structured_grid={"opacity": 0.2},
)

In [ ]:
gpv.plot_2d(
    model=geo_model,
    cell_number=2,
    override_regular_grid=geo_model.solutions.raw_arrays.mask_matrix[0],
    show_data=True,
    kwargs_lithology={"cmap": "gray", "norm": None},
)

gpv.plot_2d(
    model=geo_model,
    cell_number=2,
    override_regular_grid=geo_model.solutions.raw_arrays.mask_matrix[1],
    show_data=True,
    kwargs_lithology={"cmap": "gray", "norm": None},
)

gpv.plot_2d(
    model=geo_model,
    cell_number=2,
    override_regular_grid=geo_model.solutions.raw_arrays.mask_matrix[2],
    show_data=True,
    kwargs_lithology={"cmap": "gray", "norm": None},
)

gpv.plot_2d(
    model=geo_model,
    cell_number=2,
    override_regular_grid=geo_model.solutions.raw_arrays.mask_matrix[3],
    show_data=True,
    kwargs_lithology={"cmap": "gray", "norm": None},
)

In [ ]:
gpv.plot_2d(
    model=geo_model,
    cell_number=2,
    override_regular_grid=geo_model.solutions.raw_arrays.mask_matrix_squeezed[0],
    show_data=True,
    kwargs_lithology={"cmap": "gray", "norm": None},
)

gpv.plot_2d(
    model=geo_model,
    cell_number=2,
    override_regular_grid=geo_model.solutions.raw_arrays.mask_matrix_squeezed[1],
    show_data=True,
    kwargs_lithology={"cmap": "gray", "norm": None},
)

gpv.plot_2d(
    model=geo_model,
    cell_number=2,
    override_regular_grid=geo_model.solutions.raw_arrays.mask_matrix_squeezed[2],
    show_data=True,
    kwargs_lithology={"cmap": "gray", "norm": None},
)

gpv.plot_2d(
    model=geo_model,
    cell_number=2,
    override_regular_grid=geo_model.solutions.raw_arrays.mask_matrix_squeezed[3],
    show_data=True,
    kwargs_lithology={"cmap": "gray", "norm": None},
)

第3张图（`onlap_series`）对应的掩膜矩阵中，所有数值都是一样的（恒定值），而其他图的数值有变化。

以下是详细的解释，结合了可视化原理和 GemPy 的内部逻辑：

### 1. 可视化原理（为什么是灰色？）
你使用的绘图代码中包含了 `kwargs_lithology={"cmap": "gray", "norm": None}`。
*   **图 1、2、4（黑白图）**：这些矩阵中同时包含 `0`（False，黑色）和 `1`（True，白色）。绘图库会将最小值 0 映射为黑，最大值 1 映射为白，所以你能看到清晰的黑白分界。
*   **图 3（全灰图）**：这个矩阵中的数值很可能**全都是 1**（或者全都是 0）。当数据的最大值等于最小值时，`matplotlib` 无法进行颜色拉伸，默认会显示色谱中间的颜色——也就是**灰色**。

### 2. 地质建模逻辑（为什么数值是恒定的？）
这反映了 GemPy 在处理**超覆（Onlap）**关系时的内部计算逻辑：

*   **其他系列（Seafloor, Right, Left）**：
    它们的掩膜（Mask）反映了最终的有效区域。例如，它们都被最顶层的 `seafloor` 侵蚀了，所以矩阵的上半部分（海平面以上）是 `0`（黑色），下半部分是 `1`（白色）。

*   **Onlap Series（第3张图）**：
    *   `onlap_series` 在这个模型中是 `right_series` 进行超覆沉积的**基底面**。
    *   在 GemPy 的 `raw_arrays` 计算阶段，为了计算超覆层（`right_series`）在何处接触基底，算法可能需要 `onlap_series` 的标量场在**整个计算域**内都保持“激活”或“有效”状态。
    *   因此，在这个特定的原始数组（`mask_matrix_squeezed`）中，它的掩膜没有被 `seafloor` 切割，而是保持了**全 1（全真）**的状态。

**总结**：
灰色的图意味着 `onlap_series` 的掩膜在当前查看的数组中是**全域有效**的（全为 1），没有像其他层那样被切除。这通常是超覆关系计算过程中的中间状态。